In [2]:

import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [3]:

# 获取栅格的顶角坐标
def get_raster_extent(raster):
    desc = arcpy.Describe(raster)
    extent = desc.extent
    return [extent.XMin,extent.YMax]

# 获取栅格的像元数量
def get_raster_size(raster):
    desc = arcpy.Describe(raster)
    return [desc.width,desc.height]

# 调整栅格范围一致
def reshape_raster(stand_raster,in_raster):
    """stand_raster:参考栅格,in_raster:待调整栅格"""
    stand_raster = Raster(stand_raster)
    in_raster= Raster(in_raster)
    with arcpy.EnvManager(snapRaster=stand_raster, extent="MAXOF", cellSize=stand_raster):
        output_raster = Con(stand_raster>0,in_raster)
    return output_raster
# 调整栅格size一致
def reshape_raster_size(stand_raster,in_raster,cell_size):
    """stand_raster:参考栅格,in_raster:待调整栅格,cell_size:像元大小"""
    stand_raster = Raster(stand_raster)
    env.extent = stand_raster
    in_raster= Raster(in_raster)
    stander_array = arcpy.RasterToNumPyArray(stand_raster)
    in_array = arcpy.RasterToNumPyArray(in_raster)
    result_array = resize_arrays(stander_array,in_array,0)[1]
    # array 转 raster
    return mask_raster(result_array,stand_raster,cell_size)
# 按掩膜提取
def mask_raster(in_raster,mask_ele,size_path):
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    in_raster = Raster(in_raster)
    mask_ele = Raster(mask_ele)
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele,cellSize=size_path):
        result_raster = arcpy.sa.ExtractByMask(in_raster, mask_ele, "INSIDE")
        return result_raster

In [4]:
# 输出路径
out_path = r"D:\ArcgisData\pred_soildeep\pre_database\pred_soildeep_gz.gdb"

In [5]:
# 按掩膜提取
env.workspace = r"D:\ArcgisData\pred_soildeep\pre_database\pred_soildeep.gdb"
mask_ele = r"D:\ArcgisData\basedata\soildeep.gdb\round_polygon_Raster_p"
cell_size = r"D:\ArcgisData\basedata\basetrain_30m.gdb\DEM"

In [5]:
arcpy.ListRasters()

['NDVI', 'TMP', 'PRE', 'AP', 'NIGHT', 'SOILQS']

In [6]:
for one_raster in arcpy.ListRasters():
    mask_raster(one_raster,mask_ele,cell_size).save(f"{one_raster}_ym")

In [8]:
# 读取并检查栅格形状尺寸是否一致
raster_path = r"D:\ArcgisData\pred_soildeep\pre_database\pred_soildeep.gdb"
env.workspace = raster_path
raster_list = arcpy.ListRasters()
raster_list = [_ for _ in raster_list if _[-2:]=='ym']
raster_list

['NDVI_ym', 'TMP_ym', 'PRE_ym', 'AP_ym', 'NIGHT_ym', 'SOILQS_ym']

In [9]:
# 导出栅格数组
for one_raster in raster_list:
    temp_index = get_raster_extent(one_raster)
    temp_size = get_raster_size(one_raster)
    print(one_raster)
    print(temp_index,temp_size)
    # temp_array = pd.DataFrame(arcpy.RasterToNumPyArray(one_raster))
    # temp_array.to_csv(os.path.join(out_path,f"{one_raster}.csv"))

NDVI_ym
[58416.702553160896, 3220372.9676713645] [19678, 16001]
TMP_ym
[58416.702553160896, 3220372.9676713645] [19678, 16001]
PRE_ym
[58416.702553160896, 3220372.9676713645] [19678, 16001]
AP_ym
[58416.702553160896, 3220372.9676713645] [19678, 16001]
NIGHT_ym
[58416.702553160896, 3220372.9676713645] [19678, 16001]
SOILQS_ym
[58416.702553160896, 3220372.9676713645] [19678, 16001]
